In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
#Import dataframes 

df_player_stats_100pos = pd.read_csv('DataFiles/player_stats_100pos.csv')
df_bio = pd.read_csv('DataFiles/df_bio.csv')
data_index = pd.read_csv('DataFiles/data_index.csv')

In [3]:
columns_to_delete = ['PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
                     'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
                     'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
                     'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
                     'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
                     'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
                     'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK']

# Delete the specified columns
df_player_stats_100pos.drop(columns=columns_to_delete, inplace=True, errors='ignore')

df_player_stats_100pos.rename(columns={'Season_type': 'Season'}, inplace=True)

In [5]:
# Columns to be dropped in 'df_bio' table:
df_bio.drop(columns=['PLAYER_HEIGHT', 'PLAYER_HEIGHT_INCHES', 'PLAYER_WEIGHT', 'PTS', 'REB', 'AST'], inplace=True, errors='ignore')

In [6]:
# Merge 'df_player_stats' and ‘df_bio‘ on common keys ‘PLAYER_ID‘ and ‘Year‘ in order to get 'COUNTRY' column.
merged_df_stats_bio = pd.merge(df_player_stats_100pos, df_bio, on=['PLAYER_ID', 'Year', 'Season'], how='inner')

columns_to_delete_merged = ['GP_y','NET_RATING','OREB_PCT','DREB_PCT','USG_PCT', 'TS_PCT','AST_PCT']

merged_df_stats_bio.drop(columns=columns_to_delete_merged, inplace=True)      

In [7]:
# Checking the result of 'merged_df_stats_bio'
merged_df_stats_bio.shape

(18282, 42)

In [8]:
# Merging 'merged_df_stat_bio' with player index table on player-id to get player´s role/position.
final_merged_df = pd.merge(merged_df_stats_bio, data_index, left_on='PLAYER_ID', right_on='PERSON_ID', how='inner')

# Dropping all duplicate columns with trailing letter 'y'.
columns_to_drop = [col for col in final_merged_df.columns if 'y' in col]
final_merged_df.drop(columns=columns_to_drop, inplace=True)

# Dropping redundant columns
columns_to_drop_2 = [
    'PLAYER_LAST_NAME',
    'PLAYER_FIRST_NAME',
    'PLAYER_SLUG',
    'TEAM_ID',
    'TEAM_SLUG',
    'IS_DEFUNCT',
    'TEAM_NAME',
    'NICKNAME',
    'TEAM_ABBREVIATION',
    'ROSTER_STATUS',
    'STATS_TIMEFRAME', 
    'PERSON_ID',
    'TEAM_CITY'
]

final_merged_df.drop(columns=columns_to_drop_2, inplace=True)

In [9]:
# Drop '_x' at the end of column names 
for column in final_merged_df.columns: 
    if '_x' in column: 
        column_name = column.replace('_x', '')
        final_merged_df.rename(columns={column:column_name}, inplace=True)

# Change regular season name at 'Season' column  
final_merged_df['Season'] = final_merged_df['Season'].apply(lambda x:'Regular Season' if x =='Regular%20Season' else x)

In [10]:
# Output of final dataframe
nj = final_merged_df[(final_merged_df['PLAYER_NAME'] == 'Nikola Jokic')]
nj

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,Year,Season,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,FROM_YEAR,TO_YEAR
15457,203999,Nikola Jokic,1610612743,DEN,21.0,80,31,49,0.388,49.7,8.8,17.2,0.512,0.8,2.4,0.333,4.4,5.4,0.811,5.2,10.9,16.1,5.4,3.0,2.3,1.4,1.6,6.0,5.7,22.8,2015-16,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15458,203999,Nikola Jokic,1610612743,DEN,22.0,73,37,36,0.507,47.8,11.6,20.0,0.578,1.1,3.3,0.324,4.4,5.3,0.825,5.0,11.9,16.8,8.4,4.0,1.4,1.3,1.2,5.0,5.5,28.6,2016-17,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15459,203999,Nikola Jokic,1610612743,DEN,23.0,75,43,32,0.573,49.0,10.1,20.3,0.499,2.2,5.6,0.396,5.3,6.3,0.850,3.9,12.2,16.1,9.2,4.2,1.8,1.2,0.9,4.3,7.6,27.8,2017-18,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15460,203999,Nikola Jokic,1610612743,DEN,24.0,80,53,27,0.663,48.7,12.0,23.5,0.511,1.6,5.3,0.307,5.6,6.8,0.821,4.4,12.4,16.8,11.3,4.8,2.1,1.1,1.3,4.4,8.1,31.2,2018-19,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15461,203999,Nikola Jokic,1610612743,DEN,24.0,14,7,7,0.500,51.1,12.1,24.0,0.506,2.0,5.1,0.393,6.1,7.2,0.846,5.0,11.8,16.7,10.8,3.3,1.5,1.2,1.0,5.0,7.4,32.4,2018-19,Playoffs,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15462,203999,Nikola Jokic,1610612743,DEN,25.0,73,46,27,0.630,49.3,11.9,22.6,0.528,1.7,5.4,0.314,5.2,6.4,0.817,3.5,11.5,15.0,10.8,4.8,1.8,0.9,0.7,4.7,7.7,30.8,2019-20,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15463,203999,Nikola Jokic,1610612743,DEN,25.0,19,9,10,0.474,50.5,12.9,24.9,0.519,3.1,7.1,0.429,4.8,5.7,0.835,2.7,10.9,13.6,7.9,4.5,1.5,1.1,1.1,4.9,7.0,33.7,2019-20,Playoffs,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15464,203999,Nikola Jokic,1610612743,DEN,26.0,72,47,25,0.653,48.9,14.4,25.4,0.566,1.8,4.7,0.388,6.7,7.7,0.868,4.0,11.3,15.3,11.8,4.4,1.9,0.9,1.2,3.8,8.2,37.3,2020-21,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15465,203999,Nikola Jokic,1610612743,DEN,26.0,10,4,6,0.400,49.1,16.5,32.4,0.509,2.8,7.5,0.377,6.5,7.8,0.836,5.3,11.2,16.5,7.1,3.4,0.9,1.3,0.7,3.3,8.3,42.4,2020-21,Playoffs,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15466,203999,Nikola Jokic,1610612743,DEN,27.0,74,46,28,0.622,48.2,14.9,25.5,0.583,1.9,5.6,0.337,7.4,9.1,0.810,4.0,15.8,19.8,11.4,5.5,2.1,1.2,1.3,3.7,9.4,39.0,2021-22,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023


In [11]:
# European countries in basketball sense
europe = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
  'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
  'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
  'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 
  'United Kingdom', 'Norway', 'Switzerland', 'Belarus', 'Russia', 'Ukraine',
  'Moldova', 'Georgia', 'Armenia', 'Azerbaijan', 'Turkey', 'Israel', 'Albania',
  'Macedonia', 'Serbia', 'Montenegro', 'Bosnia and Herzegovina', 'Iceland']

In [13]:
# Function to filer on countries
def filter_europe(df, column_name='', country_list=europe):
    return df[df[column_name].isin(europe)]

# New object df for European players only
europe_df = filter_europe(final_merged_df, 'COUNTRY')
europe_df

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,Year,Season,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,FROM_YEAR,TO_YEAR
69,1059,Aleksandar Djordjevic,1610612757,POR,29.0,8,4,4,0.500,51.4,6.6,13.2,0.500,4.1,5.8,0.714,3.3,4.1,0.800,0.8,3.3,4.1,4.1,4.1,0.0,0.0,0.0,2.5,0.0,20.7,1996-97,Regular Season,NaN,Serbia,Undrafted,Undrafted,Undrafted,19,G,6-2,198.0,1996,1996
286,717,Arvydas Sabonis,1610612757,POR,32.0,69,43,26,0.623,51.3,9.6,19.2,0.498,1.4,3.8,0.371,6.5,8.4,0.777,3.3,12.6,15.9,4.3,4.4,1.8,2.4,1.0,5.9,0.0,27.0,1996-97,Regular Season,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
287,717,Arvydas Sabonis,1610612757,POR,32.0,4,1,3,0.250,53.9,9.0,21.1,0.429,1.0,4.0,0.250,3.5,4.0,0.875,4.0,9.0,13.1,4.5,4.5,1.5,1.5,2.5,9.5,4.0,22.6,1996-97,Playoffs,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
288,717,Arvydas Sabonis,1610612757,POR,33.0,73,40,33,0.548,52.1,9.1,18.5,0.493,0.7,2.6,0.261,7.2,9.1,0.798,3.3,13.0,16.3,4.9,4.2,1.5,1.8,1.0,6.0,0.0,26.1,1997-98,Regular Season,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
289,717,Arvydas Sabonis,1610612757,POR,33.0,4,1,3,0.250,53.0,9.0,19.9,0.450,0.5,1.0,0.500,6.0,7.0,0.857,3.5,11.9,15.4,3.0,5.0,3.5,1.5,3.5,9.5,0.0,24.4,1997-98,Playoffs,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18252,1631217,Moussa Diabate,1610612746,LAC,21.0,22,10,12,0.455,47.3,5.8,11.4,0.511,0.2,0.5,0.500,2.4,3.9,0.625,7.5,4.6,12.1,1.0,2.2,1.7,1.9,1.5,3.6,3.9,14.3,2022-23,Regular Season,Michigan,France,2022,2,43,25,F,6-9,210.0,2022,2023
18254,1631107,Nikola Jovic,1610612748,MIA,20.0,15,7,8,0.467,48.4,6.6,16.4,0.406,1.9,8.3,0.229,4.3,4.5,0.947,2.1,5.2,7.3,2.4,2.4,1.7,0.5,0.0,4.5,4.5,19.4,2022-23,Regular Season,NaN,Serbia,2022,1,27,5,F,6-10,205.0,2022,2023
18255,1631107,Nikola Jovic,1610612748,MIA,20.0,7,3,4,0.429,46.7,3.7,14.8,0.250,0.0,7.4,0.000,0.0,0.0,0.000,3.7,14.8,18.5,0.0,0.0,0.0,0.0,3.7,0.0,0.0,7.4,2022-23,Playoffs,NaN,Serbia,2022,1,27,5,F,6-10,205.0,2022,2023
18258,1631172,Ousmane Dieng,1610612760,OKC,20.0,39,20,19,0.513,46.7,6.2,14.9,0.420,2.1,8.0,0.265,1.2,1.9,0.652,1.6,7.1,8.6,3.8,2.1,1.1,0.6,0.7,3.4,1.6,15.8,2022-23,Regular Season,NaN,France,2022,1,11,13,F,6-9,185.0,2022,2023
